# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the file
file_path = os.path.join("..", "WeatherPy", "Output", "city_data.csv")

# Create a data frame from the CSV
city_data_df = pd.read_csv(file_path)

# Display the first few rows
city_data_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,kaka,37.35,59.61,81.05,43,0,9.93,TM,1594511988
1,rikitea,-23.12,-134.97,69.66,70,22,15.37,PF,1594511815
2,goderich,43.75,-81.72,75.00,75,69,13.27,CA,1594511896
3,banjaran,-7.05,107.59,70.68,79,76,3.24,ID,1594511989
4,juneau,58.30,-134.42,59.00,72,90,14.99,US,1594511742


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Set the figure layout with dimensions and positioning of the map
figure_layout = {'width': '1600px', 'height': '800px', 'margin': '0 auto 0 auto'}

# Create the gmaps figure with layout, set a center, set a zoom level
humidity = gmaps.figure(layout=figure_layout, center=(36.17,5.39), zoom_level=3)

# Create variables for locations (latitude and longitude) and weights (humidity)
locations = city_data_df[["Latitude", "Longitude"]]
weights = city_data_df["Humidity"]

# Create the heatmap layer, dissipating set to false stops points from changing size on zoom
heatmap = gmaps.heatmap_layer(locations, weights, dissipating=False, max_intensity=200, point_radius=3)

# Add the layer to the figure
humidity.add_layer(heatmap)

# Display the figure with the heatmap layer
humidity

Figure(layout=FigureLayout(height='800px', margin='0 auto 0 auto', width='1600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the data set to cities that meet the criteria we are searching for
vacation_df = city_data_df.loc[(city_data_df["Max Temperature"] > 70) & 
                               (city_data_df["Max Temperature"] < 80) & 
                               (city_data_df["Wind Speed"] < 10) & 
                               (city_data_df["Cloudiness"] == 0)]
vacation_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
220,heihe,50.24,127.49,77.00,69,0,4.47,CN,1594512039
265,zhangye,38.93,100.45,71.98,35,0,6.82,CN,1594512048
266,zavitinsk,50.12,129.45,75.47,79,0,3.58,RU,1594512048
291,preveza,38.95,20.75,73.40,88,0,5.82,GR,1594511969
299,sudak,44.85,34.97,71.29,67,0,9.40,UA,1594512056
356,dawlatabad,36.41,64.91,70.88,34,0,1.68,AF,1594512068
368,jalu,29.03,21.55,74.05,64,0,9.95,LY,1594511915
500,benghazi,32.12,20.07,74.66,76,0,9.57,LY,1594512096
501,lebedinyy,58.52,125.52,79.79,46,0,5.88,RU,1594512096
545,tigzirt,36.89,4.12,74.14,84,0,7.92,DZ,1594512105


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store the vacation data frame with narrowed down data in hotel_df
hotel_df = vacation_df

# Create a new column called Hotel Name
hotel_df["Hotel Name"] = ""

# Set the parameters to search for
params = {"radius": 5000,
          "keyword": "hotel",
          "key": g_key}

# Print a starting message
print("Starting search for nearby Hotels:\n")

# Set counter to 1 to offset the index
counter = 1

# Use a loop to run through the lat and long to identify nearby hotels
for index, row in hotel_df.iterrows():
    
    # Retrieve lat and long from data frame
    lat = row["Latitude"]
    lng = row["Longitude"]

    # This will change the location to the next city but leave the other parameters the same
    params["location"] = f"{lat},{lng}"

    # Store the base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make the request and convert to a json object
    hotel_location = requests.get(base_url, params=params).json()

    # Since there may be missing data, try and except to skip any missing data points
    try:
        # Add the hotel name if found to the Hotel Name column
        hotel_df.loc[index, "Hotel Name"] = hotel_location["results"][0]["name"]

        # Print message saying that hotel was found for destination number
        print(f"Hotel for destination {counter} found!")

    except (KeyError, IndexError):
        print(f"Hotel for destination {counter} not found... skipping.")

    # Increase counter for next iteration
    counter += 1

# Print ending message
print("\nSearch complete!")

Starting search for nearby Hotels:

Hotel for destination 1 found!
Hotel for destination 2 found!
Hotel for destination 3 found!
Hotel for destination 4 found!
Hotel for destination 5 found!
Hotel for destination 6 not found... skipping.
Hotel for destination 7 found!
Hotel for destination 8 found!
Hotel for destination 9 not found... skipping.
Hotel for destination 10 found!

Search complete!


In [6]:
hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
220,heihe,50.24,127.49,77.00,69,0,4.47,CN,1594512039,Mercure Blagoveshchensk
265,zhangye,38.93,100.45,71.98,35,0,6.82,CN,1594512048,Holiday Inn Express Zhangye
266,zavitinsk,50.12,129.45,75.47,79,0,3.58,RU,1594512048,Lyuks
291,preveza,38.95,20.75,73.40,88,0,5.82,GR,1594511969,daLUZ Boutique Hotel
299,sudak,44.85,34.97,71.29,67,0,9.40,UA,1594512056,Soldayya Grand Hotel
356,dawlatabad,36.41,64.91,70.88,34,0,1.68,AF,1594512068,
368,jalu,29.03,21.55,74.05,64,0,9.95,LY,1594511915,Jalu Hotel
500,benghazi,32.12,20.07,74.66,76,0,9.57,LY,1594512096,Al-Fadheel Hotel
501,lebedinyy,58.52,125.52,79.79,46,0,5.88,RU,1594512096,
545,tigzirt,36.89,4.12,74.14,84,0,7.92,DZ,1594512105,Hôtel Mizrana


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt><b>Name:</b></dt><p>{Hotel Name}</p>
<dt><b>City:</b></dt><p>{City}</p>
<dt><b>Country:</b></dt><p>{Country}</p>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
hotels = gmaps.marker_layer(locations, info_box_content=hotel_info)
humidity.add_layer(hotels)

# Display figure
humidity

Figure(layout=FigureLayout(height='800px', margin='0 auto 0 auto', width='1600px'))